In [ ]:
# import torch
# import torch.nn as nn
# import torch.functional as F
# import pytorch_lightning as pl

# from torchrbpnet.layers import Conv1DFirstLayer, Conv1DResBlock, IndexEmbeddingOutputHead
# from torchrbpnet.losses import MultinomialNLLLossFromLogits
# from torchrbpnet.data.utils import TFIterableDataset

In [ ]:
# class Network(nn.Module):
#     def __init__(self, tasks, n_layers=9, n_body_filters=256):
#         super(Network, self).__init__()

#         self.tasks = tasks

#         self.body = nn.Sequential(*[Conv1DFirstLayer(4, n_body_filters, 6)]+[(Conv1DResBlock(n_body_filters, n_body_filters, dilation=(2**i))) for i in range(n_layers)])
#         self.head = IndexEmbeddingOutputHead(len(self.tasks), dims=n_body_filters)
    
#     def forward(self, inputs, **kwargs):
#         x = inputs

#         for layer in self.body:
#             x = layer(x)

#         return self.head(x)

# network = Network(tasks=list(range(223)))

In [ ]:
# class MultiRBPNet(pl.LightningModule):
#     def __init__(self, network):
#         super().__init__()
#         self.network = network # Network(tasks=range(223))
#         self.loss_fn = MultinomialNLLLossFromLogits()

#     def training_step(self, batch, **kwargs):
#         x, y = batch
#         y_pred = self.network(x)
#         loss = self.loss_fn(y, y_pred, dim=-2)
#         return loss

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#         return optimizer

# model = MultiRBPNet(network)
# model

In [ ]:
# dataset = TFIterableDataset('example-data-matrix/windows.chr13.4.data.matrix.filtered.tfrecord', shuffle=1_000_000)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=None)

In [ ]:
import pytorch_lightning as pl

In [ ]:
from torchrbpnet.data import tfrecord_to_dataloader

dataloader = tfrecord_to_dataloader('example-data-matrix/windows.chr13.4.data.matrix.filtered.tfrecord')

In [ ]:
from torchrbpnet.networks import MultiRBPNet
from torchrbpnet.lightning import Model

In [ ]:
model = Model(network=MultiRBPNet(tasks=list(range(223))))

In [ ]:
trainer = pl.Trainer(max_epochs=2)
trainer.fit(model=model, train_dataloaders=dataloader)